In [2]:
import os  
import urllib.request 
import requests 

import datetime 
import pandas as pd
import zipfile

# Descargando datos

## Ejercicio 1
Utiliza los archivos json descargados con el detalle a nivel estatal, y genera unos 3 DataFrames con información sobre personas desaparecidas dependiendo de diferentes características.

In [74]:
# pwd
print(os.getcwd())

#  Estos son los datos que vamos a descargar y donde vamos a guardarlos
desaparecidos_RNPDNO_url = "http://www.datamx.io/dataset/fdd2ca20-ee70-4a31-9bdf-823f3c1307a2/resource/d352810c-a22e-4d72-bb3b-33c742c799dd/download/desaparecidos3ago.zip"
desaparecidos_RNPDNO_archivo = "desaparecidosRNPDNO.zip"
desaparecidos_corte_nacional_url = "http://www.datamx.io/dataset/fdd2ca20-ee70-4a31-9bdf-823f3c1307a2/resource/4865e244-cf59-4d39-b863-96ed7f45cc70/download/nacional.json"
desaparecidos_corte_nacional_archivo = "desaparecidos_nacional.json"
subdir = "./data/"

c:\Users\DELL\Documents\Areas\MCD\1S\Ingeniería de Características\ingesta de datos\Ingesta-de-datos-ejercicios


In [75]:
if not os.path.exists(desaparecidos_RNPDNO_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(desaparecidos_RNPDNO_url, subdir + desaparecidos_RNPDNO_archivo)  
    with zipfile.ZipFile(subdir + desaparecidos_RNPDNO_archivo, "r") as zip_ref:
        zip_ref.extractall(subdir)
    
    urllib.request.urlretrieve(desaparecidos_corte_nacional_url, subdir + desaparecidos_corte_nacional_archivo)  

    with open(subdir + "info.txt", 'w') as f:
        f.write("Archivos sobre personas desaparecidas\n")
        info = """
        Datos de desaparecidos, corte nacional y desagregación a nivel estatal, 
        por edad, por sexo, por nacionalidad, por año de desaparición y por mes
        de desaparición para los últimos 12 meses.

        Los datos se obtuvieron del RNPDNO con fecha de 03 de agosto de 2021
        (la base de datos no se ha actualizado últimamente) 

        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + desaparecidos_RNPDNO_url + "\n")
        f.write("Nombre: " + desaparecidos_RNPDNO_archivo + "\n")
        f.write("Agregados nacionales descargados desde: " + desaparecidos_corte_nacional_url + "\n")
        f.write("Nombre: " + desaparecidos_corte_nacional_archivo + "\n")

In [23]:
import json
id_Sonora = 26
with open(f'data/datos_procesados/estados/{id_Sonora}.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

#print(json.dumps(data['anual'], indent=4))
anual = pd.read_json(json.dumps(data['anual']))
espacial = pd.read_json(json.dumps(data['espacial']))
edades = pd.read_json(json.dumps(data['por_edad']))

C:\Users\DELL\AppData\Local\Temp\ipykernel_10792\2354727175.py:7: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  anual = pd.read_json(json.dumps(data['anual']))
C:\Users\DELL\AppData\Local\Temp\ipykernel_10792\2354727175.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  espacial = pd.read_json(json.dumps(data['espacial']))
C:\Users\DELL\AppData\Local\Temp\ipykernel_10792\2354727175.py:9: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  edades = pd.read_json(json.dumps(data['por_edad']))
C:\Users\DELL\AppData\Local\Temp\ipykernel_10792\2354727175.py:9: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing

In [24]:
anual.describe()

,Hombre,Indeterminado,Mujer
count,33.000000,33.000000,33.000000
mean,131.666667,0.212121,67.363636
std,201.476529,0.739830,104.885896
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,7.000000,0.000000,10.000000
75%,191.000000,0.000000,92.000000
max,681.000000,4.000000,423.000000


In [25]:
espacial.describe()

,Hombres,Indeterminado,Mujeres
count,64.000000,64.000000,64.000000
mean,67.890625,0.109375,34.734375
std,157.942838,0.758386,100.087778
min,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000
50%,6.000000,0.000000,2.000000
75%,55.500000,0.000000,14.000000
max,970.000000,6.000000,644.000000


In [26]:
edades.describe()

,Hombres,Indeterminado,Mujeres
count,91.000000,91.000000,91.000000
mean,47.747253,0.076923,24.428571
std,84.509775,0.340939,62.633705
min,1.000000,0.000000,0.000000
25%,6.000000,0.000000,1.000000
50%,27.000000,0.000000,5.000000
75%,81.000000,0.000000,20.000000
max,757.000000,2.000000,485.000000


## Ejercicio 2
Entender la estructura del archivo xml de poetas, hacer un query de otro tema que consideren interesante y generar un DataFrame con la información más importante. No olvides de comentar tu código y explicar la estructura del archivo xml

In [4]:
%pip install requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.5.0-py3-none-any.whl (129 kB)
Using cached certifi-2025.8.3-py3-none-any.whl (161 kB)

   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   ---------------------------------------- 0/5 [urllib3]
   -------- ------------------------------- 1/5 [idna]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ---------------- ----------------------- 2/5 [charset_normalizer]
   ------------------------ --------------- 3/5 [certifi]
   -------------------------------- ------- 4/5 [

In [6]:
import requests

# URL de la API de MediaWiki de Wikipedia en español
API_URL = "https://es.wikipedia.org/w/api.php"

# Cabecera para identificar nuestra aplicación (buena práctica)
HEADERS = {
    'User-Agent': 'WikiXMLPseudoDump/1.0 (a225230115@unison.mx)'
}

def get_page_titles(category_title):
    """Obtiene la lista de títulos de páginas de una categoría."""
    titles = []
    cmcontinue = None 
    while True:
        params = {
            "action": "query",
            "format": "json",
            "list": "categorymembers",
            "cmtitle": category_title,
            "cmlimit": "500",  # El límite máximo por petición
            "cmcontinue": cmcontinue
        }     
        response = requests.get(API_URL, params=params, headers=HEADERS)
        data = response.json()
        
        for member in data['query']['categorymembers']:
            titles.append(member['title'])           
        if 'continue' in data:
            cmcontinue = data['continue']['cmcontinue']
        else:
            break           
    return titles

def get_page_content_in_xml(page_titles):
    """Obtiene el contenido de las páginas en formato XML."""
    params = {
        "action": "query",
        "format": "xml",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "titles": "|".join(page_titles)
    }
    response = requests.get(API_URL, params=params, headers=HEADERS)
    return response.content


In [ ]:
categoria = "Académicos e investigadores de la Universidad de Sonora"
#Otros interesantes son "Escritores de Sonora", "Alumnado de la Universidad de Sonora"
    
print(f"Obteniendo la lista de entradas de '{categoria}'...")
titles = get_page_titles('Categoría:'+categoria)

print(f"Se encontraron {len(titles)} entradas.")
if not titles:
  raise ValueError("No se encontraron páginas en la categoría especificada.")

batch_size = 50 # Lote de títulos para la segunda petición (máximo 50)
all_xml_data = []

for i in range(0, len(titles), batch_size):
  batch_titles = titles[i:i + batch_size]
  xml_content = get_page_content_in_xml(batch_titles)
  all_xml_data.append(xml_content)
  print(f"Procesadas entradas {i + 1} a {i + batch_size} de {len(titles)}")

Obteniendo la lista de entradas de 'Académicos e investigadores de la Universidad de Sonora'...
Se encontraron 16 entradas.
Procesadas entradas 1 a 50 de 16


In [ ]:
import xml.etree.ElementTree as et
# Combinar todos los XML en un solo archivo
root = et.Element(categoria.lower().replace(" ", "_"))
for xml_data in all_xml_data:
  # El contenido XML de la API tiene un elemento <api> y dentro <query>, que necesitamos para el contenido
  api_root = et.fromstring(xml_data)
  query_element = api_root.find('query')
  if query_element:
    # Los elementos <page> son los que contienen la información de cada poeta
    for page_element in query_element.findall('pages/page'):
      root.append(page_element)
tree = et.ElementTree(root)

# Guardamos en un archivo con el mismo nombre que la categoría (con .xml)
output_filename = categoria.lower().replace(" ", "_") + ".xml"
with open(output_filename, "wb") as f:
      # Escribe el XML completo al archivo
        tree.write(f, encoding='utf-8', xml_declaration=True)
        print(f"\nArchivo '{output_filename}' creado exitosamente con la información completa.")


Archivo 'académicos_e_investigadores_de_la_universidad_de_sonora.xml' creado exitosamente con la información completa.


C:\Users\DELL\AppData\Local\Temp\ipykernel_23352\1469790014.py:8: DeprecationWarning: Testing an element's truth value will always return True in future versions.  Use specific 'len(elem)' or 'elem is not None' test instead.
  if query_element:


In [27]:
import re
def extract_ficha_data(text):
    start = text.find("{{Ficha de ")
    if start == -1:
        return None

    i = start + 2  # skip starting {{
    brace_count = 2
    while i < len(text) and brace_count > 0:
        if text[i:i+2] == "{{":
            brace_count += 2
            i += 2
        elif text[i:i+2] == "}}":
            brace_count -= 2
            i += 2
        else:
            i += 1

    block = text[start:i]
    
    # Extract key-value pairs
    fields = re.findall(r'\| *([^=|]+)\s*=\s*(.*)', block)
    return fields

In [71]:
docentes = et.parse(output_filename)
nombres = []
escuelas = []
fechasNacimientos= []
for docente in docentes.getroot():
    #print(docente.attrib['title'])
    data = extract_ficha_data(docente[0][0][0][0].text) #para obtener el texto del articulo de wikipedia
    nombres.append(docente.attrib['title']) #nombre del articulo de wikipedia
    if data:
        #print(data)
        escuela_no_encontrada = True
        fecha_no_encontrada = True
        for key, value in data:
            if key.strip() == "alma máter": #para obtener dodne estudiaron
                escuela_no_encontrada = False
                escuelas.append(value.strip().replace("[[","").replace("]]",""))
                #print(f"    {key.strip()}: {value.strip().replace("[[","").replace("]]","")}")
            if key.strip() == "fecha de nacimiento":#para obtener cuando nacieron
                fecha_no_encontrada = False
                fechasNacimientos.append(value.strip().replace("[[","").replace("]]",""))
        if escuela_no_encontrada:
            escuelas.append(None)
        if fecha_no_encontrada:
            fechasNacimientos.append(None)
    else:
        escuelas.append(None) #manejar las datos faltantes
        fechasNacimientos.append(None)
[nombres, escuelas,fechasNacimientos]
#[len(nombres), len(escuelas), len(fechasNacimientos)] #verificar output

[['Alfonso López Corral',
  'Anna Ivette Rodríguez Navarro',
  'Daniel Avechuco Cabrera',
  'Elizabeth Araux Sánchez',
  'Elizabeth Cejudo Ramos',
  'Emma Susana Speratti Piñero',
  'Françoise Pérus',
  'Germán Viveros Maldonado',
  'Judith Tanori Córdova',
  'Martha Bracho',
  'Martha Guzmán Partida',
  'María Rita Plancarte Martínez',
  'Miguel Manríquez Durán',
  'Olivia Gutu Ocampo',
  'Tristana Landeros',
  'Zarina Estrada Fernández'],
 [None,
  'COLSON, Universidad de Sonora',
  'Universidad de Sonora',
  None,
  'Universidad Nacional Autónoma de México. Instituto de Investigaciones Históricas/Facultad de Filosofía y Letras',
  None,
  None,
  None,
  None,
  None,
  None,
  'Universidad de Sonora',
  None,
  None,
  None,
  None],
 ['1979',
  None,
  '{{fecha|1985}}',
  None,
  '27 de junio de 1980',
  None,
  None,
  None,
  None,
  '6 de febrero de 1927',
  None,
  None,
  '7 de septiembre de 1957',
  None,
  '1974',
  None]]

In [72]:
import pandas as pd
docentesUnison = pd.DataFrame(data={'nombre':nombres,"escuela":escuelas,"fecha_nacimiento":fechasNacimientos})
docentesUnison

,nombre,escuela,fecha_nacimiento
0,Alfonso López Corral,None,1979
1,Anna Ivette Rodríguez Navarro,"COLSON, Universidad de Sonora",None
2,Daniel Avechuco Cabrera,Universidad de Sonora,{{fecha|1985}}
3,Elizabeth Araux Sánchez,None,None
4,Elizabeth Cejudo Ramos,Universidad Nacional Autónoma de México. Insti...,27 de junio de 1980
5,Emma Susana Speratti Piñero,None,None
6,Françoise Pérus,None,None
7,Germán Viveros Maldonado,None,None
8,Judith Tanori Córdova,None,None
9,Martha Bracho,None,6 de febrero de 1927


# Usando la API para obtener datos sobre personas desaparecidas del RNPDNO

Cree la siguente coleccion en Postman para explorar la API

https://www.postman.com/juanmorenoe-2526296/rnpdno/overview

y crear una mini documentacion (o por lo menos anotar lo que creo que significa cada cosa)

(Obtuve los endpoints al ver la lista de llamadas que hacia en la pagina oficial (entrando en modo desarollador `f12`) y tambien en la subpagina de estadisticas por filtro)

In [3]:
API_HOST = "https://versionpublicarnpdno.segob.gob.mx/"
API_SOCIODEOGRAFICOS_ROOT = "Sociodemografico/"
API_CATALAGO_ROOT = "Catalogo/"

ENDPOINT_CATALOGO_EDO = "Estados/"
ENDPOINT_CATALOGO_MUN = "Municipios/"
ENDPOINT_CATALOGO_COL = "Colonias/"

main_session = requests.Session()
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/Index")
main_session.get("https://versionpublicarnpdno.segob.gob.mx/Dashboard/ContextoGeneral")

<Response [200]>

In [4]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_EDO

r = main_session.post(TARGET_URL)
estados_id = pd.json_normalize(r.json(),)
estados_id.columns = ['Valor', 'Estado']
estados_id

,Valor,Estado
0,0,--TODOS--
1,1,AGUASCALIENTES
2,2,BAJA CALIFORNIA
3,3,BAJA CALIFORNIA SUR
4,4,CAMPECHE
5,7,CHIAPAS
6,8,CHIHUAHUA
7,9,CIUDAD DE MEXICO
8,5,COAHUILA
9,6,COLIMA


In [5]:
estado = 'Sonora'
idEstado = str(estados_id[estados_id['Estado']== estado.upper()]['Valor'].to_list()[0])
idEstado

'26'

In [47]:
TARGET_URL = API_HOST + API_CATALAGO_ROOT + ENDPOINT_CATALOGO_MUN
DATA = {"idEstado": idEstado}

r = main_session.post(TARGET_URL, data = DATA)
mun_id = pd.json_normalize(r.json())
mun_id.columns = ['Valor', 'Municipio']
mun_id

,Valor,Municipio
0,0,--TODOS--
1,1,ACONCHI
2,2,AGUA PRIETA
3,3,ALAMOS
4,4,ALTAR
...,...,...
69,65,TUBUTAMA
70,66,URES
71,67,VILLA HIDALGO
72,68,VILLA PESQUEIRA


In [ ]:
municipios_ids = municipio_id['Valor'].to_list()
municipios_ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 71,
 16,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 70,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 21,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 72,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 73,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69]

In [ ]:
municipio_id[municipio_id['Valor']==68]['Municipio']

72    VILLA PESQUEIRA
Name: Municipio, dtype: object

## Ejercicio 1
¿Como podríamos consultar las estadísticas sobre mujeres desaparecidas en el municipio de Cajeme?

In [48]:
municipio = 'Cajeme'
municipio_id = str(mun_id[mun_id['Municipio']== municipio.upper()]['Valor'].to_list()[0])
municipio_id

'18'

In [ ]:

TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "AreaChartSexoAnio"

DATA = {
  "idEstado":idEstado,
  "idMunicipio":municipio_id,
}
r = main_session.post(TARGET_URL, json = DATA)
datos = r.json() #como el json esta hecho para contruir una grafica de area, hay que extraer donde esta la informacion pertinente
#print(datos)
resumen_cajeme = pd.DataFrame({"Fecha":datos['XAxisCategories'],"Mujeres":datos['Series'][1]['data']})

resumen_cajeme


,Fecha,Mujeres
0,1.CIFRA SIN AÑO DE REFERENCIA,5
1,1974,0
2,1981,0
3,1998,0
4,2004,0
5,2007,1
6,2010,1
7,2011,11
8,2012,4
9,2013,10


## Ejercicio 2
¿Como podemos sacar lo que pasa en todo el estado, por municipios y por colonias? Intentalo.

In [51]:
TARGET_URL = API_HOST + API_SOCIODEOGRAFICOS_ROOT + "BarChartSexoColonia"
todas_colonias = pd.DataFrame()
for i in municipios_ids[1:]:
  DATA = {
    "titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
    "subtitulo":"POR COLONIAS",
    "idEstado":"26",
    "idMunicipio":i,
    "idColonia":"0"
  }
  r = main_session.post(TARGET_URL, json = DATA)
  res = r.json()
  datos = {serie['name']: serie['data'] for serie in res['Series']}
  por_colonia = pd.DataFrame(datos)
  por_colonia['Municipio'] = mun_id[mun_id['Valor']==i]['Municipio'].to_list()[0]
  por_colonia['Colonia'] = res['XAxisCategories']
  todas_colonias = pd.concat([todas_colonias,por_colonia], ignore_index=True)

todas_colonias

,Hombre,Mujer,Indeterminado,Municipio,Colonia
0,2.0,0.0,0.0,ACONCHI,ACONCHI
1,5.0,3.0,0.0,ACONCHI,SIN COLONIA DE REFERENCIA
2,1.0,3.0,0.0,AGUA PRIETA,VALLES DUARTE
3,5.0,7.0,0.0,AGUA PRIETA,BUENOS AIRES
4,3.0,3.0,0.0,AGUA PRIETA,PROGRESO
...,...,...,...,...,...
489,1.0,0.0,0.0,VILLA HIDALGO,SIN COLONIA DE REFERENCIA
490,2.0,2.0,0.0,VILLA PESQUEIRA,SIN COLONIA DE REFERENCIA
491,1.0,0.0,0.0,YÉCORA,GUADALUPE TOYOPA
492,8.0,2.0,0.0,YÉCORA,SIN COLONIA DE REFERENCIA


## Ejercicio 3
¿Se puede hacer por municipio? ¿En forma programática? ¿Para algun caso especial? Intentalo

In [52]:
municipio = 'Cajeme'
municipio_id = str(mun_id[mun_id['Municipio']== municipio.upper()]['Valor'].to_list()[0])
DATA = {
"titulo":"PERSONAS DESAPARECIDAS, NO LOCALIZADAS Y LOCALIZADAS",
"subtitulo":"POR COLONIAS",
"idEstado":"26",
"idMunicipio":municipio_id,
"idColonia":"0"
}
r = main_session.post(TARGET_URL, json = DATA)
res = r.json()
datos = {serie['name']: serie['data'] for serie in res['Series']}
por_colonia = pd.DataFrame(datos)
por_colonia['Colonia'] = res['XAxisCategories']
por_colonia

,Hombre,Mujer,Indeterminado,Colonia
0,1,1,0,NOROESTE
1,3,2,0,VALLE DORADO
2,1,2,0,AMANECER 1
3,1,1,0,EJIDAL
4,6,3,0,VILLAS DEL REAL
5,1,0,0,MONTECARLOS
6,1,0,0,VIVIANO MURIETA
7,1,0,0,ESPERANZA TIZNADA
8,0,2,0,LA CORTINA
9,12,5,0,VILLA BONITA


## Ejercicio 4
Extrae alguna información del conjunto de datos que pienses que es relevante, y explica porqué.